In [1]:
import sys
import pickle
# import cPickle
from datetime import datetime, timezone, timedelta
from enum import Enum
import numpy as np
import pandas as pd

from pathlib import Path
import matplotlib.colors as mcolors

# from PyQt5 import QtGui, QtWidgets
# from PyQt5.QtWidgets import QMessageBox, QToolTip, QStackedWidget, QHBoxLayout, QVBoxLayout, QSplitter, QFormLayout, QLabel, QFrame, QPushButton, QTableWidget, QTableWidgetItem, QScrollArea
# from PyQt5.QtWidgets import QApplication, QFileSystemModel, QTreeView, QWidget, QAction, qApp, QApplication, QTreeWidgetItem, QFileDialog 
# from PyQt5.QtGui import QPainter, QBrush, QPen, QColor, QFont, QIcon
# from PyQt5.QtCore import Qt, QPoint, QRect, QObject, QEvent, pyqtSignal, pyqtSlot, QSize, QDir, QThreadPool

# from GUI.UI.AbstractDatabaseAccessingWidgets import AbstractDatabaseAccessingQObject

from app.filesystem.VideoUtils import findVideoFiles, VideoParsedResults, FoundVideoFileResult, CachedFileSource
# from app.filesystem.Workers.VideoMetadataWorkers import VideoMetadataWorker, VideoMetadataWorkerSignals
# from app.filesystem.Workers.VideoFilesystemWorkers import VideoFilesystemWorker, VideoFilesystemWorkerSignals
from app.filesystem.Workers.VideoMetadataWorkers import VideoMetadataWorker
from app.filesystem.Workers.VideoFilesystemWorkers import VideoFilesystemWorker

from GUI.Model.ModelViewContainer import ModelViewContainer
from app.filesystem.FilesystemOperations import OperationTypes, PendingFilesystemOperation
from app.filesystem.LabjackEventsLoader import LabjackEventsLoader, PhoServerFormatArgs

from app.filesystem.FilesystemRecordBase import FilesystemRecordBase, FilesystemLabjackEvent_Record
from GUI.Model.Events.PhoDurationEvent import PhoDurationEvent

# from app.filesystem.LabjackFilesystemLoadingMixin import LabjackEventFile, LabjackFilesystemLoader

In [2]:
# from app.filesystem.LabjackEventsLoader import *

In [3]:
""" Import Dataframe to file:
"""
dataframe_import_path = 'data/output/LabjackDataExport/output_dataframe_1-9-2020'

print('Reading dataframe from file {}...'.format(str(dataframe_import_path)))
# out_df.to_pickle(out_dataframe_export_path)
store = pd.HDFStore('{}_store.h5'.format(str(dataframe_import_path)))
# to_hdf
in_df = store['variables_dataframe']
print('done reading dataframe to file.')
# in_df
# for (aVariableIndex, aVariableData) in enumerate(variableData):
#     aVariableData['']
        
#         print('Converting variableData to Pandas Dataframe...')
#         out_df = get_variables_as_dataframe(variableData, active_labjack_variable_names)
#         # out_df.to_json(orient='split')

Reading dataframe from file data/output/LabjackDataExport/output_dataframe_1-9-2020...
__setstate__(state: {'start_date': datetime.datetime(2019, 9, 16, 21, 0, 7, 491000), 'end_date': None, 'variable_name': 'Water1_BeamBreak', 'variable_color': <PyQt5.QtGui.QColor object at 0x000002800248E278>, 'extended_info_dict': {'videoIndex': 26, 'video_relative_offset': datetime.timedelta(0, 11198, 458000), 'event_type': 'BeamBreak', 'dispense_type': 'BeamBreak', 'port': 'Water1'}})
__setstate__(state: {'start_date': datetime.datetime(2019, 9, 16, 21, 0, 7, 796000), 'end_date': None, 'variable_name': 'Water1_BeamBreak', 'variable_color': <PyQt5.QtGui.QColor object at 0x000002800248E278>, 'extended_info_dict': {'videoIndex': 26, 'video_relative_offset': datetime.timedelta(0, 11198, 763000), 'event_type': 'BeamBreak', 'dispense_type': 'BeamBreak', 'port': 'Water1'}})
__setstate__(state: {'start_date': datetime.datetime(2019, 9, 16, 21, 0, 33, 565000), 'end_date': None, 'variable_name': 'Water1_Beam

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



timestamps  values videoIndicies  \
Water1_BeamBreak 0     2019-09-16 21:00:07.491     1.0            26   
                 1     2019-09-16 21:00:07.796     1.0            26   
                 2     2019-09-16 21:00:33.565     1.0            26   
                 3     2019-09-16 21:12:46.651     1.0            26   
                 4     2019-09-16 21:12:47.210     1.0            26   
...                                        ...     ...           ...   
Food2_Dispense   14270 2019-08-20 19:29:18.031     1.0           210   
                 14271 2019-08-20 19:29:46.095     1.0           210   
                 14272 2019-08-20 19:41:22.972     1.0           210   
                 14273 2019-08-20 19:42:24.613     1.0           210   
                 14274 2019-08-20 20:27:37.802     1.0           210   

                                                  variableSpecificRecords  
Water1_BeamBreak 0      <app.filesystem.FilesystemRecordBase.Filesyste...  
                 1      <app.filesystem.FilesystemRecordBase.Filesyste...  
                 2      <app.filesystem.FilesystemRecordBase.Filesyste...  
                 3      <app.filesystem.FilesystemRecordBase.Filesyste...  
                 4      <app.filesystem.FilesystemRecordBase.Filesyste...  
...                                                                   ...  
Food2_Dispense   14270  <app.filesystem.FilesystemRecordBase.Filesyste...  
                 14271  <app.filesystem.FilesystemRecordBase.Filesyste...  
                 14272  <app.filesystem.FilesystemRecordBase.Filesyste...  
                 14273  <app.filesystem.FilesystemRecordBase.Filesyste...  
                 14274  <app.filesystem.FilesystemRecordBase.Filesyste...  

[41189 rows x 4 columns]

In [25]:
in_df = in_df.T

In [26]:
in_df.shape

(4, 41189)

In [27]:
in_df.index

Index(['timestamps', 'values', 'videoIndicies', 'variableSpecificRecords'], dtype='object')

In [28]:
in_df.columns

MultiIndex([('Water1_BeamBreak',     0),
            ('Water1_BeamBreak',     1),
            ('Water1_BeamBreak',     2),
            ('Water1_BeamBreak',     3),
            ('Water1_BeamBreak',     4),
            ('Water1_BeamBreak',     5),
            ('Water1_BeamBreak',     6),
            ('Water1_BeamBreak',     7),
            ('Water1_BeamBreak',     8),
            ('Water1_BeamBreak',     9),
            ...
            (  'Food2_Dispense', 14265),
            (  'Food2_Dispense', 14266),
            (  'Food2_Dispense', 14267),
            (  'Food2_Dispense', 14268),
            (  'Food2_Dispense', 14269),
            (  'Food2_Dispense', 14270),
            (  'Food2_Dispense', 14271),
            (  'Food2_Dispense', 14272),
            (  'Food2_Dispense', 14273),
            (  'Food2_Dispense', 14274)],
           length=41189)

In [29]:
in_df.dtypes

Water1_BeamBreak  0        object
                  1        object
                  2        object
                  3        object
                  4        object
                            ...  
Food2_Dispense    14270    object
                  14271    object
                  14272    object
                  14273    object
                  14274    object
Length: 41189, dtype: object

In [24]:
in_df.variableSpecificRecords

Water1_BeamBreak  0        <app.filesystem.FilesystemRecordBase.Filesyste...
                  1        <app.filesystem.FilesystemRecordBase.Filesyste...
                  2        <app.filesystem.FilesystemRecordBase.Filesyste...
                  3        <app.filesystem.FilesystemRecordBase.Filesyste...
                  4        <app.filesystem.FilesystemRecordBase.Filesyste...
                                                 ...                        
Food2_Dispense    14270    <app.filesystem.FilesystemRecordBase.Filesyste...
                  14271    <app.filesystem.FilesystemRecordBase.Filesyste...
                  14272    <app.filesystem.FilesystemRecordBase.Filesyste...
                  14273    <app.filesystem.FilesystemRecordBase.Filesyste...
                  14274    <app.filesystem.FilesystemRecordBase.Filesyste...
Name: variableSpecificRecords, Length: 41189, dtype: object

In [12]:
# in_df.axes
in_df.head(20)

timestamps  values videoIndicies  \
Water1_BeamBreak 0  2019-09-16 21:00:07.491     1.0            26   
                 1  2019-09-16 21:00:07.796     1.0            26   
                 2  2019-09-16 21:00:33.565     1.0            26   
                 3  2019-09-16 21:12:46.651     1.0            26   
                 4  2019-09-16 21:12:47.210     1.0            26   
                 5  2019-09-16 23:58:38.373     1.0            27   
                 6  2019-09-16 23:58:38.678     1.0            27   
                 7  2019-09-17 00:12:02.597     1.0            27   
                 8  2019-09-17 00:12:02.902     1.0            27   
                 9  2019-09-17 04:26:00.276     1.0            28   
                 10 2019-09-17 20:24:33.003     1.0            33   
                 11 2019-09-17 20:24:33.308     1.0            33   
                 12 2019-09-17 20:24:48.285     1.0            33   
                 13 2019-09-17 20:24:48.590     1.0            33   
                 14 2019-09-17 20:25:07.125     1.0            33   
                 15 2019-09-17 21:05:11.730     1.0            33   
                 16 2019-09-17 21:05:12.294     1.0            33   
                 17 2019-09-17 21:05:20.695     1.0            33   
                 18 2019-09-17 21:05:21.001     1.0            33   
                 19 2019-09-17 21:06:07.757     1.0            33   

                                               variableSpecificRecords  
Water1_BeamBreak 0   <app.filesystem.FilesystemRecordBase.Filesyste...  
                 1   <app.filesystem.FilesystemRecordBase.Filesyste...  
                 2   <app.filesystem.FilesystemRecordBase.Filesyste...  
                 3   <app.filesystem.FilesystemRecordBase.Filesyste...  
                 4   <app.filesystem.FilesystemRecordBase.Filesyste...  
                 5   <app.filesystem.FilesystemRecordBase.Filesyste...  
                 6   <app.filesystem.FilesystemRecordBase.Filesyste...  
                 7   <app.filesystem.FilesystemRecordBase.Filesyste...  
                 8   <app.filesystem.FilesystemRecordBase.Filesyste...  
                 9   <app.filesystem.FilesystemRecordBase.Filesyste...  
                 10  <app.filesystem.FilesystemRecordBase.Filesyste...  
                 11  <app.filesystem.FilesystemRecordBase.Filesyste...  
                 12  <app.filesystem.FilesystemRecordBase.Filesyste...  
                 13  <app.filesystem.FilesystemRecordBase.Filesyste...  
                 14  <app.filesystem.FilesystemRecordBase.Filesyste...  
                 15  <app.filesystem.FilesystemRecordBase.Filesyste...  
                 16  <app.filesystem.FilesystemRecordBase.Filesyste...  
                 17  <app.filesystem.FilesystemRecordBase.Filesyste...  
                 18  <app.filesystem.FilesystemRecordBase.Filesyste...  
                 19  <app.filesystem.FilesystemRecordBase.Filesyste...